# Applying fairnes metrics on the Adult Census Dataset

<a href="https://colab.research.google.com/drive/10BcFbOU2RsKVF-o1wz40K-JNfhADE043" target="_blank">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).


A _machine Learning bias_, also sometimes called _algorithm bias_ (NOT TO BE CONFUSED WITH A MATH BIAS), is a phenomenon that occurs when an algorithm produces results that are systemically prejudiced due to erroneous assumptions in the machine learning process.

Machine Learning Fairness is an area of research that seeks to uncover the roots of such biases and how we can mitigate them. Addressing algorithmic biases is a crucial issue in ML, given that machine learning models are increasingly used in fields where their decisions can impact people's lives, such as finance, employment, and criminal justice.

<img src="https://miro.medium.com/max/1060/1*cc8OWxqKFXje4d_1eYrQkg.jpeg" width="600"/>

Source: _[Moritz Hardt](https://towardsdatascience.com/a-tutorial-on-fairness-in-machine-learning-3ff8ba1040cb/)._

In this notebook, we will be using the [`adult census dataset`](https://www.kaggle.com/datasets/uciml/adult-census-income), created by Barry Becker from the 1994 Census database (USA), to explore biases in ML algorithms. The prediction task of this dataset is to determine whether a person makes over 50K a year.

The eatures and values that can be found in the `adult census dataset` are the following:

- `Income`: '>50K', '<=50K'.
- `Age`: continuous.
- `Workclass`: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.  
- `fnlwgt`: continuous.  
- `Education`: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.  
- `Education-num`: continuous.  
- `Marital-status`: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.
- `Occupation`: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspect, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.  
- `Relationship`: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.  
- `Race`: White, Asian-Pac-Islander, Amer-Indian--Eskimo, Other, Black.  
- `Sex`: Female, Male.  
- `Capital-gain`: continuous.  
- `Capital-loss`: continuous.  
- `Hours-per-week`: continuous.  
- `Native-country`: United States, Cambodia, England, Puerto Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

This notebook will consider `Age`, `Race`, and `Sex` as sensitive attributes. The dataset can be found on the Hugging Face Hub! 🤗


In [14]:
!pip install datasets -q
from datasets import load_dataset

# load the datasets from the hub
data = load_dataset('AiresPucrs/adult-census-income', split='train')

# turn the datasets into a pandas.DataFrame
df = data.to_pandas()
display(df)

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
0,90,?,77053,HS-grad,9,Widowed,?,Not-in-family,White,Female,0,4356,40,United-States,<=50K
1,82,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
2,66,?,186061,Some-college,10,Widowed,?,Unmarried,Black,Female,0,4356,40,United-States,<=50K
3,54,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,41,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,22,Private,310152,Some-college,10,Never-married,Protective-serv,Not-in-family,White,Male,0,0,40,United-States,<=50K
32557,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32558,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32559,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K


Below, we perform some simple pre-processing (e.g., excluding rows with `?` values and grouping the age category into buckets) to help the training of our model.

In [15]:
import numpy as np
import pandas as pd

df = df.replace('?', np.nan)
df = df.dropna(axis=0).copy()

bins = [17, 27, 37, 47, 57, 67, 77, 87, 97]
labels = ['17-27','27-37','37-47','47-57','57-67','67-77','77-87','87-97']

df.loc[:, 'age'] = pd.cut(df['age'], bins=bins, labels=labels)


display(df.head())

,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,income
1,77-87,Private,132870,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,0,4356,18,United-States,<=50K
3,47-57,Private,140359,7th-8th,4,Divorced,Machine-op-inspct,Unmarried,White,Female,0,3900,40,United-States,<=50K
4,37-47,Private,264663,Some-college,10,Separated,Prof-specialty,Own-child,White,Female,0,3900,40,United-States,<=50K
5,27-37,Private,216864,HS-grad,9,Divorced,Other-service,Unmarried,White,Female,0,3770,45,United-States,<=50K
6,37-47,Private,150601,10th,6,Separated,Adm-clerical,Unmarried,White,Male,0,3770,40,United-States,<=50K


Now, let us try to see how our target (`income`) is related to the sensitive attributes of our dataset (`Age`, `Race`, and `Sex`).

In [17]:
less_50_list = []
more_50_list = []

for element in list(df['age'].unique().dropna()):
    less_50 = df[df['age'] == element]['income'].value_counts()['<=50K']
    more_50 = df[df['age'] == element]['income'].value_counts()['>50K']
    less_50_list.append(less_50)
    more_50_list.append(more_50)

import plotly.graph_objects as go

fig = go.Figure(data=[
    go.Bar(name='<=50K', x=list(df['age'].unique().dropna()), y=less_50_list),
    go.Bar(name='>50K', x=list(df['age'].unique().dropna()), y=more_50_list)
])

fig.update_layout(
    barmode='group',
    template='plotly_dark',
    xaxis_title="<b>Age-Buckets</b>",
    yaxis_title="<b>Income by Age Buckets</b>",
    title='Distribution of <i>Income</i> by "Age-Buckets"',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    )
fig.show()

less_50_list = []
more_50_list = []

for element in list(df['sex'].unique()):
    less_50 = df[df['sex'] == element]['income'].value_counts()['<=50K']
    more_50 = df[df['sex'] == element]['income'].value_counts()['>50K']
    less_50_list.append(less_50)
    more_50_list.append(more_50)

fig = go.Figure(data=[
    go.Bar(name='<=50K', x=list(df['sex'].unique()), y=less_50_list),
    go.Bar(name='>50K', x=list(df['sex'].unique()), y=more_50_list)
])

fig.update_layout(
    barmode='group',
    template='plotly_dark',
    xaxis_title="<b>Sex</b>",
    yaxis_title="<b>Income by Sex</b>",
    title='Distribution of <i>Income</i> by "Sex"',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    )
fig.show()

less_50_list = []
more_50_list = []

for element in list(df['race'].unique()):
    less_50 = df[df['race'] == element]['income'].value_counts()['<=50K']
    more_50 = df[df['race'] == element]['income'].value_counts()['>50K']
    less_50_list.append(less_50)
    more_50_list.append(more_50)

fig = go.Figure(data=[
    go.Bar(name='<=50K', x=list(df['race'].unique()), y=less_50_list),
    go.Bar(name='>50K', x=list(df['race'].unique()), y=more_50_list)
])

fig.update_layout(
    barmode='group',
    template='plotly_dark',
    xaxis_title="<b>Race</b>",
    yaxis_title="<b>Income by Race</b>",
    title='Distribution of <i>Income</i> by "Race"',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    )
fig.show()

Let us see if our sensitive attributes correlate with `Income`. If they are, this is already a sign that our future model could inherit these biases against a specific unprivileged group.

To be able to calculate correlations, let us transform all categorical values into numbers.

In [12]:
corr_df = df.copy()

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for column in list(set(df.columns) - set(df._get_numeric_data().columns)):
    corr_df[column] = le.fit_transform(corr_df[column])

import plotly.express as px

fig = px.imshow(corr_df.corr(numeric_only=True).values,
                labels=dict(x="Features", y="Features"),
                x=list(corr_df.columns),
                y=list(corr_df.columns),
                text_auto=True
                )
fig.update_xaxes(side='top')
fig.update_layout(template='plotly_dark',
                  title='Correlation Matrix',
                  coloraxis_showscale=False,
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

According to the correlation scores, `Sex` ($0.21$) and `Age` ($0.17$) have a slight positive correlation with `Income` (race has a correlation that verges 0). Hence, it is safe to assume that these features might have some influence on the classification of a system trained on this dataset.

To test this claim, let us write a simple classifier (a `dense neural network`) and evaluate it against [statistical fairness metrics](https://arxiv.org/abs/2001.07864).

In [18]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

seed = 42
x_train, x_test, y_train, y_test = train_test_split(corr_df.drop('income', axis=1).values,\
     tf.one_hot(corr_df['income'].values, 2).numpy(), test_size=0.2, random_state=seed)

regularizer = tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)

model = tf.keras.models.Sequential([tf.keras.layers.Flatten(input_shape=(x_train.shape[1],)),
                                    tf.keras.layers.Dense(
                                        64, activation='relu', kernel_regularizer=regularizer),
                                    tf.keras.layers.Dense(
                                        32, activation='relu', kernel_regularizer=regularizer),
                                    tf.keras.layers.Dense(2, activation='softmax')])

opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

model.fit(x_train, y_train, epochs=5,
                    batch_size=128, validation_split=0.2,
                    verbose=1)

test_loss_score, test_acc_score = model.evaluate(x_test, y_test)

print(f'Final Loss: {test_loss_score:.2f}.')
print(f'Final Performance: {test_acc_score * 100:.2f} %.')

predictions = model.predict(x_test, verbose=0)

from sklearn.metrics import confusion_matrix

matrix = confusion_matrix(y_test.argmax(axis=1), predictions.argmax(axis=1))

fig = px.imshow(matrix,
                labels=dict(x="Predicted", y="True label"),
                x=['<=50K', '>50K'],
                y=['<=50K', '>50K'],
                text_auto=True
                )
fig.update_xaxes(side='top')
fig.update_layout(template='plotly_dark',
                  title='Confusion Matrix',
                  coloraxis_showscale=False,
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)')
fig.show()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 14)                0         
                                                                 
 dense_3 (Dense)             (None, 64)                960       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 2)                 66        
                                                                 
Total params: 3106 (12.13 KB)
Trainable params: 3106 (12.13 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Version:  2.15.0
Eager mode:  True
GPU is NOT AVAILABLE
Epoch 1/5
151/151 [==============================] - 3s 7ms/step - loss: 574.8478 - accuracy: 0.

The network tends to get an accuracy of $\approx$ 70%. Let us see how this will change when we consider the sensitive attributes present in the dataset.

### Statistical Fairness Metrics

_Fairness metrics are a set of measures that enable you to detect the presence of bias in your model._ For a full review of the most prominent definitions of fairness in algorithmic classification, we recommend [_Fairness Definitions Explained_](https://fairware.cs.umass.edu/papers/Verma.pdf). Below, you will see how to apply these definitions.

Since all these definitons are a way to compare the rates of correct/incorrect predictions, it is worth to remember what is a confusion matrix. A confusion matrix is a table often used to describe the performance of a classification model (or "classifier") on a set of test data for which the actual values are known.

![confusion-matrix](https://upload.wikimedia.org/wikipedia/commons/3/32/Binary_confusion_matrix.jpg)

Where:

- $TP$: True Positive (_correct positive classification_, e.g., you make >50K and you are classified as `>50K`)
- $FP$: False Positive (_incorrect positive classification_, e.g., you dont make >50K an you are classified as `>50K`)
- $TN$: True Negative (_correct negative classification_, e.g., you make <=50K and you are classified as `<=50K`)
- $FN$: False Negative (_incorrect negative classification_, e.g., you dont make <=50K and you are classified as `<=50K`)

With these values, there are several ways to assess how biased an algorithm may be by comparing how certain combinations of these scores change about specific groups.

> **Note: In the literature, it is common to refer to these groups, in the context of a fairness analysis, as `privileged` or `unprivileged`.**

### Common Statistical Fairness Metrics

- **Statistical Parity Ratio:** Statistical Parity Ratio compares the proportion of members of a given group that were classified for the positive class (i.e., correctly or not, a.k.a., TP and FP) to another group (privileged versus unprivileged).

$$\frac{\text{Statistical Parity}_{\text{ unprivileged}}}{\text{Statistical Parity}_{\text{ privileged}}} = \frac{(\frac{TP  +  FP}{TP  +  FP  +  TN  +  FN})_{\text{unprivileged}}}{(\frac{TP  +  FP}{TP  +  FP  +  TN  +  FN})_{\text{privileged}}}$$

- **Equal Opportunity Ratio:** Equal Opportunity ratio compares the true positive rate (i.e., TPR, a.k.a., _Sensitivity/Recall_) of different groups (privileged versus unprivileged).

$$\frac{\text{TPR}_{\text{ unprivileged}}}{\text{TPR}_{\text{ privileged}}} = \frac{(\frac{TP}{TP+FN})_{\text{unprivileged}}}{(\frac{TP}{TP+FN})_{\text{privileged}}}$$

- **Predictive Parity Ratio:** Predictive Parity Ratio compares the positive predictive value (i.e., PPV, a.k.a., _Precision_) of different groups (privileged versus unprivileged).

$$\frac{\text{PPV}_{\text{ unprivileged}}}{\text{PPV}_{\text{ privileged}}} = \frac{(\frac{TP}{TP+FP})_{\text{unprivileged}}}{(\frac{TP}{TP+FP})_{\text{privileged}}}$$

- **Predictive Equality Ratio:** Predictive Equality Ratio compares the false positive rate (FPR, a.k.a., _fall-out/false alarm ratio_) of different groups (privileged versus unprivileged).

$$\frac{\text{FPR}_{\text{ unprivileged}}}{\text{FPR}_{\text{ privileged}}} = \frac{(\frac{FP}{FP+TN})_{\text{unprivileged}}}{(\frac{FP}{FP+TN})_{\text{privileged}}}$$

- **Accuracy Equality Ratio:** Accuracy Equality Ratio compares the proportion of members of a given group that were correctly classified (i.e., _accuracy_) to another group (privileged versus unprivileged).

$$\frac{\text{Accuracy}_{\text{ unprivileged}}}{\text{Accuracy}_{\text{ privileged}}} = \frac{(\frac{TP  +  TN}{TP  +  FP  +  TN  +  FN})_{\text{unprivileged}}}{(\frac{TP  +  TN}{TP  +  FP  +  TN  +  FN})_{\text{privileged}}}$$

- **Equalized Odds:** Equalized Odds it is the most restrictive concept of ML Fairness. This criteria is only satisfied if both groups (privileged versus unprivileged) have equal TPR and FPR.

$$\frac{\text{TPR}_{\text{ unprivileged}}}{\text{TPR}_{\text{ privileged}}} = 1 \;\land\;\frac{\text{FPR}_{\text{ unprivileged}}}{\text{FPR}_{\text{ privileged}}} = 1$$

Let us now  implement a function that calculates all these metrics.

In [19]:

def calc_fair(model, DataFrame, protected_atributte, group_priv, group_unpriv, label):
    """
    The function calc_fair computes several fairness metrics for a given machine
    learning model on a test set DataFrame. The fairness metrics calculated include
    statistical parity ratio, true positive rate, positive predictive value,
    false positive rate, accuracy, equal opportunity ratio, predictive parity
    ratio, predictive equality ratio, and accuracy equality ratio. The function
    takes in the following arguments:

    Args:
    --------
        - model: The trained machine learning model to evaluate fairness on.
        - DataFrame: The test set data used to evaluate the model.
        - protected_attribute: The name of the protected attribute in the DataFrame.
        - group_priv: The value of the protected attribute for the privileged group.
        - group_unpriv: The value of the protected attribute for the unprivileged group.
        - label: The name of the column in the DataFrame that contains the ground truth labels.

    Returns:
    --------
    The function returns a dictionary containing the fairness metric names and their
    corresponding scores. The scores are rounded to two decimal places. Additionally,
    the function returns the equalized odds as a string.
    """
    test_set = DataFrame.copy()

    for column in list(set(test_set.columns) - set(test_set._get_numeric_data().columns)):
        test_set[column] = le.fit_transform(test_set[column])

    test_set_priv_labels, test_set_priv = tf.one_hot(list(test_set[test_set[protected_atributte] == group_priv][label]), 2).numpy(), test_set[test_set[protected_atributte] == group_priv].drop(label, axis = 1).values
    test_set_unpriv_labels, test_set_unpriv = tf.one_hot(list(test_set[test_set[protected_atributte] == group_unpriv][label]), 2).numpy(), test_set[test_set[protected_atributte] == group_unpriv].drop(label, axis = 1).values

    preds_priv = model.predict(test_set_priv, verbose=0)
    preds_unpriv = model.predict(test_set_unpriv, verbose=0)

    TN_PV, FP_PV, FN_PV, TP_PV = confusion_matrix(test_set_priv_labels.argmax(axis=1), preds_priv.argmax(axis=1)).ravel()
    TN_UPV, FP_UPV, FN_UPV, TP_UPV = confusion_matrix(test_set_unpriv_labels.argmax(axis=1), preds_unpriv.argmax(axis=1)).ravel()

    statistical_parity_priv = (TP_PV + FP_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # STATISTICAL PARITY RATIO
    statistical_parity_unpriv = (TP_UPV + FP_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # STATISTICAL PARITY RATIO
    equal_oportunity_priv = TP_PV / (TP_PV+FN_PV)  # TRUE POSITIVE RATIO
    equal_oportunity_unpriv = TP_UPV / (TP_UPV+FN_UPV)  # TRUE POSITIVE RATIO
    predictive_parity_priv = TP_PV/(TP_PV + FP_PV)  # POSITIVE PREDICTIVE VALUE
    predictive_parity_unpriv = TP_UPV/(TP_UPV + FP_UPV)  # POSITIVE PREDICTIVE VALUE
    predictive_equality_priv = FP_PV / (FP_PV+TN_PV)  # FALSE POSITIVE RATE
    predictive_equality_unpriv = FP_UPV / (FP_UPV+TN_UPV)  # FALSE POSITIVE RATE
    accuracy_equality_priv = (TP_PV + TN_PV)/(TP_PV + FP_PV + TN_PV + FN_PV)  # ACCURACY EQUALITY RATIO
    accuracy_equality_unpriv = (TP_UPV + TN_UPV)/(TP_UPV + FP_UPV + TN_UPV + FN_UPV)  # ACCURACY EQUALITY RATIO

    if statistical_parity_priv >= statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_unpriv/statistical_parity_priv
    elif statistical_parity_priv < statistical_parity_unpriv:
        statistical_parity_ratio = statistical_parity_priv/statistical_parity_unpriv

    if equal_oportunity_priv >= equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_unpriv/equal_oportunity_priv
    elif equal_oportunity_priv < equal_oportunity_unpriv:
        equal_oportunity_ratio = equal_oportunity_priv/equal_oportunity_unpriv

    if predictive_parity_priv >= predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_unpriv/predictive_parity_priv
    elif predictive_parity_priv < predictive_parity_unpriv:
        predictive_parity_ratio = predictive_parity_priv/predictive_parity_unpriv

    if predictive_equality_priv >= predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_unpriv/predictive_equality_priv
    elif predictive_equality_priv < predictive_equality_unpriv:
        predictive_equality_ratio = predictive_equality_priv/predictive_equality_unpriv

    if accuracy_equality_priv >= accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_unpriv/accuracy_equality_priv
    elif accuracy_equality_priv < accuracy_equality_unpriv:
        accuracy_equality_ratio = accuracy_equality_priv/accuracy_equality_unpriv

    data = {'Fairness Metrics': ['Chance of receiving the positive class - privileged',
                                'Chance of receiving the positive class - unprivileged',
                                'Statistical Parity Ratio (SPR)',
                                'True Positive Rate - privileged',
                                'True Positive Rate - unprivileged',
                                'Equal Opportunity Ratio (EOR)',
                                'Positive Predictive Value - privileged',
                                'Positive Predictive Value - unprivileged',
                                'Predictive Parity Ratio (PPR)',
                                'False Positive Rate - privileged',
                                'False Positive Rate - unprivileged',
                                'Predictive Equality Ratio (PER)',
                                'Accuracy - privileged',
                                'Accuracy - unprivileged',
                                'Accuracy Equality Ratio (AER)',
                                'Equalized Odds'],
            'Scores': [round(statistical_parity_priv, 2),
                        round(statistical_parity_unpriv, 2),
                        round(statistical_parity_ratio,2),
                        round(equal_oportunity_priv, 2),
                        round(equal_oportunity_unpriv, 2),
                        round(equal_oportunity_ratio, 2),
                        round(predictive_parity_priv,2),
                        round(predictive_parity_unpriv,2),
                        round(predictive_parity_ratio,2),
                        round(predictive_equality_priv,2),
                        round(predictive_equality_unpriv,2),
                        round(predictive_equality_ratio,2),
                        round(accuracy_equality_priv,2),
                        round(accuracy_equality_unpriv,2),
                        round(accuracy_equality_ratio,2),
                        f'TPR: {round(equal_oportunity_priv, 2)} vs {round(equal_oportunity_unpriv, 2)}. FPR: {round(predictive_equality_priv,2)} vs {round(predictive_equality_unpriv,2)}']
            }
    return pd.DataFrame(data).set_index('Fairness Metrics')

Let us use the feature `race` as an example. Remember that, after encoding, `race` feature values are:

- $4$ = `White`
- $3$ = `Other`
- $2$ = `Black`
- $1$ = `Asian-Pac-Islander`
- $0$ = `Amer-Indian-Eskimo`

Let us define $4$, the group with more samples, as the privileged, and $2$ as the unprivileged group.

In [20]:
from IPython.display import Markdown
fairness_df = calc_fair(model, df, 'race', 4, 2, 'income')

display(Markdown(fairness_df.to_markdown()))

| Fairness Metrics                                      | Scores                             |
|:------------------------------------------------------|:-----------------------------------|
| Chance of receiving the positive class - privileged   | 0.02                               |
| Chance of receiving the positive class - unprivileged | 0.01                               |
| Statistical Parity Ratio (SPR)                        | 0.49                               |
| True Positive Rate - privileged                       | 0.06                               |
| True Positive Rate - unprivileged                     | 0.07                               |
| Equal Opportunity Ratio (EOR)                         | 0.95                               |
| Positive Predictive Value - privileged                | 0.83                               |
| Positive Predictive Value - unprivileged              | 0.89                               |
| Predictive Parity Ratio (PPR)                         | 0.94                               |
| False Positive Rate - privileged                      | 0.0                                |
| False Positive Rate - unprivileged                    | 0.0                                |
| Predictive Equality Ratio (PER)                       | 0.27                               |
| Accuracy - privileged                                 | 0.75                               |
| Accuracy - unprivileged                               | 0.88                               |
| Accuracy Equality Ratio (AER)                         | 0.85                               |
| Equalized Odds                                        | TPR: 0.06 vs 0.07. FPR: 0.0 vs 0.0 |

Depending on the fairness metric we choose, we see different results. For example, with a **Statistical Parity Ratio** of $0.55$, the privileged class is almost $2$ times as likely to be classified as the positive class (if the positive class was `>50k`). Also, even though **Accuracy Equality Ratio** (AER) is above $80\%$, the model is $10\%$ less accurate against the unprivileged group. **Equal Opportunity Ratio** (EOR), **Predictive Parity Ratio** (PPR), and **Equalized Odds** are inside "_acceptable ranges_" (i.e., above 80%).

> **Note: The results may vary depending on your classifier and how you have trained it.**

Interestingly enough, these results do not appear when we, for example, measure group $4$ (`White`) against group $1$ (`Asian-Pac-Islander`). All metrics are well above acceptable thresholds.


In [21]:
from IPython.display import Markdown
fairness_df = calc_fair(model, df, 'race', 4, 1, 'income')

display(Markdown(fairness_df.to_markdown()))

| Fairness Metrics                                      | Scores                             |
|:------------------------------------------------------|:-----------------------------------|
| Chance of receiving the positive class - privileged   | 0.02                               |
| Chance of receiving the positive class - unprivileged | 0.02                               |
| Statistical Parity Ratio (SPR)                        | 0.88                               |
| True Positive Rate - privileged                       | 0.06                               |
| True Positive Rate - unprivileged                     | 0.08                               |
| Equal Opportunity Ratio (EOR)                         | 0.77                               |
| Positive Predictive Value - privileged                | 0.83                               |
| Positive Predictive Value - unprivileged              | 1.0                                |
| Predictive Parity Ratio (PPR)                         | 0.83                               |
| False Positive Rate - privileged                      | 0.0                                |
| False Positive Rate - unprivileged                    | 0.0                                |
| Predictive Equality Ratio (PER)                       | 0.0                                |
| Accuracy - privileged                                 | 0.75                               |
| Accuracy - unprivileged                               | 0.75                               |
| Accuracy Equality Ratio (AER)                         | 0.99                               |
| Equalized Odds                                        | TPR: 0.06 vs 0.08. FPR: 0.0 vs 0.0 |

Fairness metrics can help us determine if our model discriminates according to any particular definition of fairness. However, as an [impossibility theorem](https://arxiv.org/abs/2007.06024), some Fairness metrics (_Statistical Parity_, _Equalized Odds_, _Predictive Parity_) are incompatible and cannot be completely satisfied simultaneously. Thus, the choice of which metric to use must be made according to the context of an application (i.e., benefit awarding, medical diagnosis, etc.).

---

Return to the [castle](https://github.com/Nkluge-correa/teeny-tiny_castle).